In [8]:
import pandas as pd
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key

In [9]:
# create a set of 2000 random lat and long combinations
lats = np.random.uniform(low = -90, high = 90, size = 2000)
lngs = np.random.uniform(low = -180, high = 180, size = 2000)
lat_lngs = zip(lats, lngs)

In [10]:
# add lats and longs to list
coordinates = list(lat_lngs)

In [11]:
# city list
cities = []

# gets nearest city for each lat/long combo
for coords in coordinates:
    city = citipy.nearest_city(coords[0], coords[1]).city_name
    
    # if the city is unique, then add to list
    if city not in cities:
        cities.append(city)
        
# print count to confirm in sufficient
len(cities)

754

In [12]:
# basic url for openweather API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [13]:
# city data list
city_data = []

# loop through cities
for i, city in enumerate(cities):
    
    # url for each city
    city_url = url + '&q=' + city.replace(' ', '+')
    
    # api request for each city
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        
        # dictionary inside a list inside a dictionary
        city_description = city_weather['weather'][0]['description']
        
        # Append the city information into city_data list.
        city_data.append({'City': city.title(),
                          'Country': city_country,
                          'Lat': city_lat,
                          'Lng': city_lng,
                          'Max Temp': city_max_temp,
                          'Humidity': city_humidity,
                          'Cloudiness': city_clouds,
                          'Wind Speed': city_wind,
                          'Current Description': city_description})

    # If an error is experienced, skip the city.
    except:
        pass

In [14]:
# check city_data
len(city_data)

686

In [15]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Lattes,FR,43.57,3.90,69.80,64,79,3.36,broken clouds
1,Turtas,RU,58.94,69.13,56.16,87,1,3.09,clear sky
2,Nekrasovskoye,RU,57.68,40.37,41.00,100,0,3.96,clear sky
3,Rikitea,PF,-23.12,-134.97,71.85,81,38,8.19,scattered clouds
4,Purranque,CL,-40.92,-73.17,37.40,100,66,2.44,broken clouds
5,Kavieng,PG,-2.57,150.80,83.89,72,34,7.92,light rain
6,Ushuaia,AR,-54.80,-68.30,32.00,90,40,8.05,scattered clouds
7,Pevek,RU,69.70,170.31,44.06,71,53,12.30,broken clouds
8,Qaanaaq,GL,77.48,-69.36,44.04,81,100,5.23,light rain
9,Vaini,TO,-21.20,-175.20,73.40,100,90,12.75,moderate rain


In [16]:
# output file
output = 'WeatherPy_Database.csv'

# export to csv
city_data_df.to_csv(output, index_label = 'City_ID')